In [1]:
import torch

In [2]:
import logging
from hydra import compose, initialize
from omegaconf import OmegaConf

from autoeye.config import Config

if __name__ == "__main__":
    # global initialization
    initialize(version_base=None, config_path="conf", job_name="test_app")
    cfg = compose(config_name="config")
    print(OmegaConf.to_yaml(cfg))
    log = logging.getLogger(__name__)
    Config.setup(cfg, log, train=True)

data:
  csv_files:
  - ./data/train.csv
  - ./data/test.csv
model:
  name: autoeye
  backbone: resnet
  version: 1.0
optim:
  beta1: 0.9
  beta2: 0.95
  weight_decay: 0.1
hyper:
  pretrained: true
  batch_size: 8
  dropout: 0.2
  lr: 0.0003
  epochs: 10
  eval_iters: 1
  use_amp: false



In [3]:
from autoeye.model import AutoEye


model = AutoEye()
checkpoint = torch.load("./models/autoeye_resnet.pt")
model.load(checkpoint)

/home/spectre/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

Using cache found in /home/spectre/.cache/torch/hub/pytorch_vision_v0.10.0
/home/spectre/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/spectre/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
model.backbone = resnet

In [8]:
checkpoint = {
    "backbone": model.backbone.state_dict(),
    "classifier": model.classifier.state_dict(),
    "optimizer": checkpoint["optimizer"],
    "scaler": checkpoint["scaler"],
    "epochs": 0,
}

In [9]:
torch.save(checkpoint, "./models/autoeye_resnet.pt")

In [3]:
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_vits14

Using cache found in /home/spectre/.cache/torch/hub/facebookresearch_dinov2_main
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.0+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.10 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/home/spectre/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/spectre/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/spectre/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warni

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [4]:
torch.save(dinov2_vits14, "./models/dino.pt")

In [5]:
dino_model = torch.load("./models/dino.pt")
dino_model

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [2]:
import torch
target = torch.nn.functional.one_hot(torch.arange(0, 2))[1]
target

tensor([0, 1])

In [21]:
sub_classes = torch.nn.functional.one_hot(torch.arange(0, 4), num_classes=4)
sub_classes

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

In [17]:
y = torch.nn.functional.one_hot(torch.arange(0, 5))[4].unsqueeze(0)
y

tensor([[0, 0, 0, 0, 1]])

In [18]:
torch.argmax(y, 1)

tensor([4])